In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from classification_models.tfkeras import Classifiers
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks,regularizers
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
test_pd = pd.read_csv('/kaggle/input/original-dataset/Dataset/HAM10000/ISIC2018_Task3_Test_GroundTruth.csv')
test_pd.head()

In [ ]:
train_df = pd.read_csv('/kaggle/input/dataset-train-test-new/Data-model_traindf/Dataset_model_split45_clean.csv')
vali_df = pd.read_csv('/kaggle/input/ham10000-split45/HAM1000_DF_S45/vali_df_split45.csv')

In [ ]:
vali_df.head()

In [ ]:
# Image id of train and test images
train_list = list(train_df['image_id'])
vali_list = list(vali_df['image_id'])

In [ ]:
test_list = list(test_pd['image_id'])

In [ ]:
len(vali_list)

In [ ]:
len(train_list)

In [ ]:
# Set the image_id as the index in data_pd
train_df.set_index('image_id', inplace=True)
vali_df.set_index('image_id', inplace=True)

In [ ]:
# Set the image_id as the index in data_pd
test_pd.set_index('image_id', inplace=True)

In [ ]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [ ]:
train_path = '/kaggle/input/dataset-resnet-train20000/train_dir'
vali_path = '/kaggle/input/dataset-resnet-train20000/vali_dir'
test_path = '/kaggle/input/dataset-resnet-train20000/test_dir'
batch_size = 16

In [ ]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

In [ ]:
image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nVali Batches: ")
vali_batches =datagen.flow_from_directory(vali_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)
print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)

In [ ]:
#Soft Attention

from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
import tensorflow as tf



class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x

        
        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
    
        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]
        
        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))

        
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1) 
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)

        
        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)       
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))
   
            x_exp = K.expand_dims(x,axis=-2)
   
            u = kl.Multiply()([exp_softmax_alpha, x_exp])   
  
            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])   

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u
        
        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape): 
        return [self.out_features_shape, self.out_attention_maps_shape]

    
    def get_config(self):
        return super(SoftAttention,self).get_config()
 

In [ ]:
ResNet34, preprocess_input = Classifiers.get('resnet18')

resnet= ResNet34(include_top=True,
    weights="imagenet",
    input_shape=(224,224,3),
    classes=1000,)

In [ ]:
# for i, layer in enumerate(resnet.layers):
#     layer.trainable = False
#     print(f'Layer {i}: {layer.name} - Trainable: {layer.trainable}')

In [ ]:
# #-33, -88, -125
# for layer in resnet.layers[-125:]:
#     layer.trainable = True

In [ ]:
conv = resnet.layers[-4].output

Soft Attention

In [ ]:

attention_layer2,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer2=MaxPooling2D(pool_size=(2,2), padding='same')(attention_layer2)
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(conv)

concat2=concatenate([maxpool,attention_layer2])
conv = Activation("relu")(concat2)
conv= Dropout(0.5)(conv)

In [ ]:
output = GlobalAveragePooling2D()(conv)
output = Dense(1024, activation='relu', kernel_regularizer=regularizers.l1(0.01))(output)
output = BatchNormalization()(output)

In [ ]:
output = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(output)
output = BatchNormalization()(output)

In [ ]:
output = Dense(7, activation='softmax')(output)
model = Model(inputs=resnet.input, outputs=output)

In [ ]:
model.summary()

In [ ]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.001,epsilon=0.01)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
class_weights = {   
                    0: 1,  # akiec 1
                    1: 1,  # bcc 1
                    2: 1,  # bkl 1
                    3: 1,  # df 1 
                    4: 5,  # mel 5
                    5: 1,  # nv 1
                    6: 1,  # vasc 1
                }


checkpoint=  ModelCheckpoint(filepath = 'ResNet18pre_SA_TEST_DTS20000_2201.hdf5',monitor='val_loss',save_best_only=True,save_weights_only=True)


In [ ]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=10, min_delta=0.001)
with tf.device('/gpu'):
    history = model.fit(train_batches,
                    steps_per_epoch=(len(train_df)/10),
                    epochs=150,
                    verbose=2,
                    validation_data=vali_batches,validation_steps=len(vali_df)/batch_size,callbacks=[checkpoint,Earlystop],class_weight=class_weights)

In [ ]:
from tensorflow.keras import models
model.load_weights("/kaggle/working/ResNet18pre_SA_TEST_DTS20000_2201.hdf5")

In [ ]:
with tf.device('/gpu'):
    predictions = model.predict(test_batches, steps=len(test_pd)/batch_size, verbose=0)

In [ ]:
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image 
y_true = test_batches.classes
#getting the predicted labels per image 
y_prob=predictions
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_true)

# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report:")
print(report)

In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)
#print(cm)
sns.heatmap(cm, annot=True)